In [ ]:
# default_exp step

# Step

> *Incremental Collaborative Filtering* algorithms.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import numpy as np
import torch.tensor as tensor

from tqdm import tqdm

from cf_step.networks import SimpleCF

In [ ]:
# export
class Step:
    def __init__(self, model, objective, optimizer):
        self.model = model
        self.objective = objective
        self.optimizer = optimizer
    
    @property
    def user_embeddings(self):
        return self.model.user_embeddings
    
    @property
    def item_embeddings(self):
        return self.model.item_embeddings
        
    def batch_fit(self, data_loader, epochs=1):
        for epoch in range(epochs):
            with tqdm(total=len(data_loader)) as pbar:
                for _, (users, items, ratings, preferences) in enumerate(data_loader):
                    predictions = self.model(users, items)
                    loss = self.objective(predictions, tensor(1)).mean()
                    loss.backward()
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    pbar.update(1)
    
    def step(self, user, item, rating=None, preference=None):
        prediction = self.model(user, item)
        loss = self.objective(prediction, 1)
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()
        
    def predict(self, user, k):
        user_embedding = self.user_embeddings(user)
        item_embeddings = self.item_embeddings.weight
        score = item_embeddings @ user_embedding.transpose(0, 1)
        predictions = score.squeeze().argsort()[-k:]
        return predictions